In [1]:
import arcpy, pandas as pd
from arcpy import env
from arcpy.sa import *
from otherfunctions import folders_exist
from datetime import date, timedelta

In [2]:
# Paths to input datasets
root_folder = r"Z:\PhD_Datasets&Analysis\Info_Inputs"
tam_out_dir = r"Z:\PhD_Datasets&Analysis\Outputs\T&M_WBM"
tc_ds = root_folder + "\\TerraClimate"
out_geotiff = tc_ds + "\\GeoTIFF"

# Set arcpy environment variables
env.overwriteOutput = True
arcpy.CheckOutExtension("spatial")
# env.cellSize = "MINOF" # Avoided to prevent huge files
env.cellSize = out_geotiff + "\\ppt_2023_1.tif" # Use TerraClimate resolution as reference for cell size
env.workspace = r"Z:\PhD_Datasets&Analysis\_ProcessingCache"
env.outputCoordinateSystem = arcpy.SpatialReference("WGS 1984") # WGS 1984 (4326)

In [3]:
# Get the current environment's spatial reference
spatial_ref = env.outputCoordinateSystem

# Check if a spatial reference is set
if spatial_ref:
    print(f"Spatial Reference Name: {spatial_ref.name}")
    print(f"Spatial Reference WKID: {spatial_ref.factoryCode}")
else:
    print("No spatial reference is set in the current environment.")

Spatial Reference Name: GCS_WGS_1984
Spatial Reference WKID: 4326


In [4]:
# Read the recession constant (k) raster
k = Float(Raster(r"Z:\PhD_Datasets&Analysis\Inputs\k_recession_all.tif"))

In [5]:
######################################################
### Starting values for the water balance model - T&M
######################################################

# TerraClimate available period
terra_st_yr = 1958
terra_ed_yr = 2023
sdate = date(terra_st_yr, 1, 1) # start date
edate = date(terra_ed_yr, 12, 31) # end date

# Base flow of the previous month (mm) (bf0)
bflow_ant = 10

In [9]:
period_dates = [(sdate + timedelta(days=i)).strftime('%Y-%m-%d') for i in range((edate - sdate).days + 1)]
period_dates

['1958-01-01',
 '1958-01-02',
 '1958-01-03',
 '1958-01-04',
 '1958-01-05',
 '1958-01-06',
 '1958-01-07',
 '1958-01-08',
 '1958-01-09',
 '1958-01-10',
 '1958-01-11',
 '1958-01-12',
 '1958-01-13',
 '1958-01-14',
 '1958-01-15',
 '1958-01-16',
 '1958-01-17',
 '1958-01-18',
 '1958-01-19',
 '1958-01-20',
 '1958-01-21',
 '1958-01-22',
 '1958-01-23',
 '1958-01-24',
 '1958-01-25',
 '1958-01-26',
 '1958-01-27',
 '1958-01-28',
 '1958-01-29',
 '1958-01-30',
 '1958-01-31',
 '1958-02-01',
 '1958-02-02',
 '1958-02-03',
 '1958-02-04',
 '1958-02-05',
 '1958-02-06',
 '1958-02-07',
 '1958-02-08',
 '1958-02-09',
 '1958-02-10',
 '1958-02-11',
 '1958-02-12',
 '1958-02-13',
 '1958-02-14',
 '1958-02-15',
 '1958-02-16',
 '1958-02-17',
 '1958-02-18',
 '1958-02-19',
 '1958-02-20',
 '1958-02-21',
 '1958-02-22',
 '1958-02-23',
 '1958-02-24',
 '1958-02-25',
 '1958-02-26',
 '1958-02-27',
 '1958-02-28',
 '1958-03-01',
 '1958-03-02',
 '1958-03-03',
 '1958-03-04',
 '1958-03-05',
 '1958-03-06',
 '1958-03-07',
 '1958-03-

In [10]:
# Create folders for other variables of tam model
bflow_dir = tam_out_dir + '\\bflow'
folders_exist([bflow_dir])

# Folder with percolation rasters resulting from the model
perc_dir = tam_out_dir + '\\perc'